In [1]:
import cantera as ct
import os 
import re
from rmgpy.molecule import Molecule
from rmgpy.reaction import Reaction
from rmgpy.species import Species
from rmgpy.data.kinetics.family import KineticsFamily
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.data.base import Database
from rmgpy.chemkin import load_species_dictionary

Make a species dictionary

In [2]:
unneeded_species = ['CHF(T)',
'CF2(T)',
'CH3CF(T)',
'CH2FCH(T)',
'CH2FCF(T)',
'CHF2CH(S)',
'CHF2CF(T)',
'CF3CH(S)',
'CF3CF(T)',
'CF3CO']


species_dictionary = {}


file = './hcof_anl0_with_c2_20231126.yaml'

with open(file, 'r') as f: 
    lines = f.readlines()
#species are in lines 12-103
for line in lines[11:103]:
    split_species = line.split(', #')
    [label, smiles] = split_species
    
    #handle the label first
    label = label.replace(' ', '')
        
    #now the smiles
    smiles = smiles.replace(' ', '').replace('\n', '').replace(',','')
    match=re.search('([a-z]+)', smiles)
    if match:
        also_remove = match.group(1)
        smiles = smiles.replace(also_remove, '')
        
    #now save to species dictionary
    if label in unneeded_species: 
        continue
    else:
        species_dictionary[label] = smiles
        

In [3]:
#add the last couple of stragglers

species_dictionary['H']= "[H]"
species_dictionary['H2']= "[H][H]"
species_dictionary['CH']= "[CH]"
species_dictionary['CH2']= "[CH2]"
species_dictionary['CH3']= "[CH3]"
species_dictionary['CH4']= "C"
species_dictionary['O']= "[O]"
species_dictionary['OH']= "[OH]"
species_dictionary['H2O']= "O"
species_dictionary['O2']= "[O][O]"
species_dictionary['HO2']= "[O]O"
species_dictionary['H2O2']= "OO"
species_dictionary['CO']= "[C-]#[O+]"
species_dictionary['CO2']= "O=C=O"
species_dictionary['HCO']= "[CH]=O"
species_dictionary['CH2O'] = "C=O"

Now let's make the reaction.py

But we need the degeneracy first

In [4]:
gas = ct.Solution('/work/westgroup/nora/Code/projects/PFAS/ESSCI/models/ANL_Brown5/format_reactions/Brown_difluoromethane_/hcof_anl0_with_c2_20231126.yaml')

In [5]:
pdep_rxns ={}
non_pdep = {}
for ind, x in enumerate(gas.reactions()):
    if ind<81:
        continue
    try: 
        x.rates
        pdep_rxns[ind] = x
    except TypeError as e: 
        non_pdep[ind] = x

/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
Property 'rates' to be removed after Cantera 2.6.
This property is moved to the PlogRate object accessed via the 'rate' property.
  import sys


In [ ]:
#H-abstraction 0-40(including)
#F-abstraction 41-80
#Substitution 81, 82, 83, 84
#will have to make a Brown halogens pdep library too

let's make the pdeps into a library called "Brown_pdep"

In [153]:
list(non_pdep.values())

[CH4 + F <=> CH3F + H    <Reaction(Arrhenius)>,
 CH3F + F <=> CH2F2 + H    <Reaction(Arrhenius)>,
 CH2F2 + F <=> CHF3 + H    <Reaction(Arrhenius)>,
 CHF3 + F <=> CF4 + H    <Reaction(Arrhenius)>,
 CH2 + F <=> CHF + H    <Reaction(Arrhenius)>,
 CH2 + F <=> CH + HF    <Reaction(Arrhenius)>,
 CHF + F <=> CF2 + H    <Reaction(Arrhenius)>,
 CHF + F <=> CF + HF    <Reaction(Arrhenius)>,
 CHF + O <=> CF + OH    <Reaction(Arrhenius)>,
 CHF + O <=> FCO + H    <Reaction(Arrhenius)>,
 CF + OH <=> FCO + H    <Reaction(Arrhenius)>,
 FCO + H <=> CO + HF    <Reaction(Arrhenius)>,
 CHF2 + O <=> CF2 + OH    <Reaction(Arrhenius)>,
 CHF2 + O <=> CF2O + H    <Reaction(Arrhenius)>,
 CHF2 + O <=> FCO + HF    <Reaction(Arrhenius)>,
 CF2 + OH <=> CF2O + H    <Reaction(Arrhenius)>,
 CF2 + OH <=> FCO + HF    <Reaction(Arrhenius)>,
 FCO + OH <=> CO2 + HF    <Reaction(Arrhenius)>,
 CF2O + H2O <=> CFOOH + HF    <Reaction(Arrhenius)>]

In [6]:
continued_count = 203 #want continued count to start at 0
non_pdep_list = list(non_pdep.values())
for rxn in non_pdep_list:
    continued_count+=1
    equation_string = str(rxn)
    rate = rxn.rate


    if len(rxn.reactants)==1 and list(rxn.reactants.values())[0]==1.0:
        A = "{:e}".format(rate.pre_exponential_factor)
        Ea = rate.activation_energy/(1000*1000)
        n = rate.temperature_exponent
        Arrhenius_string = f"Arrhenius(A=({A},'s^-1'), n={n}, Ea=({Ea},'kJ/mol'), T0=(1,'K'))"

    else: 
        A = "{:e}".format(rate.pre_exponential_factor*(100*100*100)/1000)
        Ea = rate.activation_energy/(1000*1000)
        n = rate.temperature_exponent
        Arrhenius_string = f"Arrhenius(A=({A},'cm^3/mol/s'), n={n}, Ea=({Ea},'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'))"

    
    f_string = f'''
    entry(
    index = {continued_count},
    label = "{equation_string}",
    degeneracy = 1.0,
    kinetics = {Arrhenius_string},
    rank = 5,
    shortDesc = """From Brown difluoromethane mech""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
    )
    '''
    print(f_string)
 



    entry(
    index = 204,
    label = "CH4 + F <=> CH3F + H",
    degeneracy = 1.0,
    kinetics = Arrhenius(A=(4.570000e+04,'cm^3/mol/s'), n=2.58, Ea=(84.9,'kJ/mol'), T0=(1,'K'), Tmin=(300,'K')),
    rank = 5,
    shortDesc = """From Brown difluoromethane mech""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
    )
    

    entry(
    index = 205,
    label = "CH3F + F <=> CH2F2 + H",
    degeneracy = 1.0,
    kinetics = Arrhenius(A=(1.470000e+04,'cm^3/mol/s'), n=2.61, Ea=(87.4,'kJ/mol'), T0=(1,'K'), Tmin=(300,'K')),
    rank = 5,
    shortDesc = """From Brown difluoromethane mech""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
    )
    

    entry(
    index = 206,
    label = "CH2F2 + F <=> CHF3 + H",
    degeneracy = 1.0,
    kinetics = Arrhenius(A=(1.620000e+03,'cm^3/mol/s'), n=2.73, Ea=(116.8,'kJ/mol'), T0=(1,'K'), Tmin=(300,'K

In [157]:
used = []
non_pdep_list = list(non_pdep.values())

for rxn in non_pdep_list:
    reactants=list(rxn.reactants.keys())
    products=list(rxn.products.keys())
    involved_species = reactants + products
    for spec in involved_species:
        if spec not in used:
            used.append(spec)


In [158]:
for spec in used:
    molecule = Molecule(smiles=species_dictionary[spec])
    print(spec)
    print(molecule.to_adjacency_list())


CH4
1 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}
5 H u0 p0 c0 {1,S}

F
multiplicity 2
1 F u1 p3 c0

CH3F
1 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2 F u0 p3 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}
5 H u0 p0 c0 {1,S}

H
multiplicity 2
1 H u1 p0 c0

CH2F2
1 F u0 p3 c0 {2,S}
2 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}
3 F u0 p3 c0 {2,S}
4 H u0 p0 c0 {2,S}
5 H u0 p0 c0 {2,S}

CHF3
1 F u0 p3 c0 {2,S}
2 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}
3 F u0 p3 c0 {2,S}
4 F u0 p3 c0 {2,S}
5 H u0 p0 c0 {2,S}

CF4
1 F u0 p3 c0 {2,S}
2 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}
3 F u0 p3 c0 {2,S}
4 F u0 p3 c0 {2,S}
5 F u0 p3 c0 {2,S}

CH2
multiplicity 3
1 C u2 p0 c0 {2,S} {3,S}
2 H u0 p0 c0 {1,S}
3 H u0 p0 c0 {1,S}

CHF
multiplicity 3
1 C u2 p0 c0 {2,S} {3,S}
2 F u0 p3 c0 {1,S}
3 H u0 p0 c0 {1,S}

CH
multiplicity 2
1 C u1 p1 c0 {2,S}
2 H u0 p0 c0 {1,S}

HF
1 F u0 p3 c0 {2,S}
2 H u0 p0 c0 {1,S}

CF2
multiplicity 3
1 F u0 p3 c0 {2,S}
2 C u2 p0 c0 {1,S} {3,S}
3 F u0 p3 c0 {2,

In [6]:
#existing H abstraction training dictionary 
e_path = '/home/khalil.nor/Code/RMG-database/input/kinetics/families/H_Abstraction/training/dictionary.txt'
existing_dict = load_species_dictionary(e_path)

brown_path = './species_dictionary_edited.txt'
brown_dict = load_species_dictionary(brown_path)

In [89]:
o = get_labeled_adj_lists(gas.reactions()[0])

for k,v in o.items():
    print(k)
    print(v[0])

CH3F
1    F u0 p3 c0 {2,S}
2 *1 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}
3 *2 H u0 p0 c0 {2,S}
4    H u0 p0 c0 {2,S}
5    H u0 p0 c0 {2,S}

H
multiplicity 2
1 *3 H u1 p0 c0

CH2F
multiplicity 2
1    F u0 p3 c0 {2,S}
2 *3 C u1 p0 c0 {1,S} {3,S} {4,S}
3    H u0 p0 c0 {2,S}
4    H u0 p0 c0 {2,S}

H2
1 *1 H u0 p0 c0 {2,S}
2 *2 H u0 p0 c0 {1,S}

